## ETL Project

##### This is high school data and student personal data for the ETL project. The data was cleaned by creating a database then extracting specific data for freshmens in MySQL. The data contains over 30k records. 

In [1]:
#Dependencies
import pandas as pd
from sqlalchemy import create_engine
from config import mysql_un, mysql_pw, mysql_uri, mysql_port


In [2]:
#Read in csv for student data
student = "students_info.csv"
student_df = pd.read_csv(student)
student_df.head()

,Student ID,student_name,gender,ethincity,date_of_birth,age,by_lunch,primary_language
0,0,Paul Bradley,M,African_American,1/29/03,16,Yes,Chinese
1,1,Victor Smith,M,Other,6/4/03,16,No,Arabic
2,2,Kevin Rodriguez,M,Asian,4/17/03,16,No,English
3,3,Dr. Richard Scott,M,White,7/24/01,18,Yes,Spanish
4,4,Bonnie Ray,F,Hispanic,9/18/02,16,No,Arabic


In [3]:
#Read in csv for hs grade levels data
grades = "high_school_grades.csv"
grades_df = pd.read_csv(grades)
grades_df.head()

,Student ID,grade,school_name,reading_score,math_score
0,0,9th,Huang High School,66,79
1,1,12th,Huang High School,94,61
2,2,12th,Huang High School,90,60
3,3,12th,Huang High School,67,58
4,4,9th,Huang High School,97,84


In [4]:
#Create DataFrame for student data

# Create a filtered dataframe from specific columns
student_df_cols = ['Student ID','student_name', 'gender', 'age']
student_transformed = student_df[student_df_cols].copy()

# Rename the column headers
student_transformed = student_transformed.rename(columns={"Student ID": "ID","student_name": "name"})

student_transformed.head()

,ID,name,gender,age
0,0,Paul Bradley,M,16
1,1,Victor Smith,M,16
2,2,Kevin Rodriguez,M,16
3,3,Dr. Richard Scott,M,18
4,4,Bonnie Ray,F,16


#### Transforming high_school_grades into a DataFrame

In [5]:
#Create DataFrame for hs grade level data

# Create a filtered dataframe from specific columns
grades_df_cols = ['Student ID','grade','school_name','reading_score', 'math_score']
grades_transformed = grades_df[grades_df_cols].copy()

# Rename the column headers
grades_transformed = grades_transformed.rename(columns={"Student ID": "ID","school_name": "school"})

grades_transformed.head()

,ID,grade,school,reading_score,math_score
0,0,9th,Huang High School,66,79
1,1,12th,Huang High School,94,61
2,2,12th,Huang High School,90,60
3,3,12th,Huang High School,67,58
4,4,9th,Huang High School,97,84


In [6]:
#Load DataFrames to MySQL 
from config import mysql_un, mysql_pw, mysql_uri, mysql_port

# connection_string = f"mysql://{mysql_un}:{mysql_pw}@{mysql_uri}:{mysql_port}/snow_report"
# connection_string = "root:Denver2018?!@localhost/dataSet_db"

connection_string = f"{mysql_un}:{mysql_pw}@{mysql_uri}/hsdataSet_db"
engine = create_engine(f'mysql://{connection_string}')

In [7]:
# Verfiy table names
engine.table_names()

[]

In [8]:
#Pandas is used to load student DataFrame to MySQL 
student_transformed.to_sql(name='students', con=engine, if_exists='append', index=False)

In [9]:
#Pandas is used to load hs level grade DataFrame to MySQL 
grades_transformed.to_sql(name='grades', con=engine, if_exists='append', index=False)

In [10]:
# Query student table to verify students data uploaded
pd.read_sql_query('select * from students', con=engine).head()

,ID,name,gender,age
0,0,Paul Bradley,M,16
1,1,Victor Smith,M,16
2,2,Kevin Rodriguez,M,16
3,3,Dr. Richard Scott,M,18
4,4,Bonnie Ray,F,16


In [11]:
# Query student table to verify students data uploaded
pd.read_sql_query('select * from grades', con=engine).head()

,ID,grade,school,reading_score,math_score
0,0,9th,Huang High School,66,79
1,1,12th,Huang High School,94,61
2,2,12th,Huang High School,90,60
3,3,12th,Huang High School,67,58
4,4,9th,Huang High School,97,84


In [12]:
#Join tables to get freshmen reading and math scores
pd.read_sql_query('select * from students s left join grades g on s.id = g.id where grade like "9th"', con=engine).head()

,ID,name,gender,age,ID,grade,school,reading_score,math_score
0,0,Paul Bradley,M,16,0,9th,Huang High School,66,79
1,4,Bonnie Ray,F,16,4,9th,Huang High School,97,84
2,5,Bryan Miranda,M,16,5,9th,Huang High School,94,94
3,12,Brittney Walker,F,16,12,9th,Huang High School,64,79
4,13,William Long,M,17,13,9th,Huang High School,71,79
